## Bedoeld om ook de overall waarden te berekenen

In [1]:
%reload_ext autoreload
%autoreload 2
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.io.wavfile as wv
import scipy.signal as sig
import wave

from datetime import datetime
from keras.models import Sequential, load_model
from keras.layers import Dense

from util import util
from util import WavFileParts
from util.logUtil import LOG, LOG_HEADER
from util.confusionMatrix import ConfusionMatrix


Using TensorFlow backend.


### globale settings

In [2]:
classes = ['music', 'voice', 'environment']

macDir = '/Volumes/SAA_DATA/datasets/'
winDir = 'E:/SAA_DATA/'
osDir = winDir
recordingDir = osDir + '/localizationRecordings'

if osDir == winDir:
    storageFolder = 'E:/SAA_DATA/storedData/'
else:
    storageFolder = '/Users/etto/Desktop/storedData/'

baseSrcDir = osDir + 'localizationFiles/20171025AllExtractionsMic4'
orgWavDirs1 = ['G428_0.0_1.4',
              'G527_0.5_1.4',
              'Studio_2.0_4.2'
              ]

orgWavDirs2 = ['G428_2.1_2.4',
              'G527_1.2_5.8',
              'Studio_3.0_2.0'
              ]

orgsG428 = ['G428_0.0_1.4','G428_2.1_2.4']
orgsG527 = ['G527_0.5_1.4','G527_1.2_5.8']
orgsStudio = ['Studio_2.0_4.2','Studio_3.0_2.0']

NFFT = 1024

chunksBaseDir = 'chunks'
rooms = ['Studio', 'G428', 'G527']

### utility functies

In [3]:
def readSoundChunksDynamic(moduleString):
    chunks = importlib.import_module(moduleString).soundChunks
    wfPts = []
    for jsonString in chunks:
        wfPts.append(WavFileParts.WavFilePartFromJson(jsonString))
    return wfPts

def timeFunction(func):
    """
    Aanroep: bijv. fpc = timeFunction(lambda: getFilesPerCategory(srcDir))
    """
    startTime = datetime.now()
    print('Start: ' + startTime.strftime('%H:%M:%S') + '\n=================')

    res = func()
    
    endTime = datetime.now()
    print('\n=================\nEnd: ' + endTime.strftime('%H:%M:%S'))
    print('Time taken: '),
    print(endTime - startTime)
    print()
    
    return res
    
def storeTestData(allSpectros, allClasses, storageName, keyName):
    filename = storageFolder + storageName + '.hd5'
    df = pd.DataFrame(allSpectros)
    df.to_hdf(path_or_buf=filename, key='spectros_' + keyName)

    df = pd.DataFrame(allClasses)
    df.to_hdf(path_or_buf=filename, key='classes_' + keyName)

def retrieveTestData(storageName, keyName):
    filename = storageFolder + storageName + '.hd5'
    specDf = pd.read_hdf(path_or_buf=filename, key='spectros_' + keyName)
    classesDf = pd.read_hdf(path_or_buf=filename, key='classes_' + keyName)
    return specDf.values, classesDf.values
    

### functies tbv trainen

In [4]:
# Maakt een dictionary aan; per categorie alle files (volledig pad) uit de srcDir
# srcDir is een van de orgWavDirs, bijvoorbeeld
#    localizationFiles/20171025AllExtractionsMic4/G428_0.0_1.4
def getFilesPerCategory(srcDir):
    filesPerCategory = {}
    for catDirLong in glob.glob(srcDir + '/*'):
        catDir = catDirLong.replace('\\', '/')
        catDir = catDir.replace(srcDir + '/', '')

        filesPerCategory[catDir] = []
        for filename in glob.glob(catDirLong + '/*'):
            filename = filename.replace('\\','/')
            filesPerCategory[catDir].append(filename)
    return filesPerCategory

def getFilesPerCatFromMultipleDirs(srcDirs, srcDirsBase=''):
    filesPerCat = {}
    for dirName in srcDirs:
        srcDir = srcDirsBase + '/' + dirName
        fpcNw = getFilesPerCategory(srcDir)
        if not filesPerCat:
            filesPerCat = fpcNw
        else:
            for key in filesPerCat:
                filesPerCat[key] += fpcNw[key]
    return filesPerCat
        

In [5]:
# Maakt een dictionary aan; per categorie de spectrogrammen
def getSpectrosFromFilesPerCategory(filesPerCategory):
    spectros = {}
    for clz in classes:
        spectros[clz] = []
        for filename in filesPerCategory[clz]:
            fs, signal = wv.read(filename)
            freq_array, segment_times, spectrogram = sig.spectrogram(x=signal, fs=fs, nfft=NFFT, noverlap=0)
            spectros[clz].append(spectrogram.T)
    return spectros

In [6]:
def getClassLengths(spectrosPerCat):
    clzLengths = {}
    for clz in classes:
        clzLengths[clz] = sum([np.shape(lst)[0] for lst in spectrosPerCat[clz]])
    return clzLengths

In [7]:
# verwacht invoer van getSpectrosFromFilesPerCategory
# levert traindata op (X_train en Y_train)
def createTrainDataFromSpectros(spectrosPerCat, clzLengths):
    X_train = np.concatenate(spectrosPerCat[classes[0]], axis=0)
    for i in range(1, len(classes)):
        nwSpectros = np.concatenate(spectrosPerCat[classes[i]], axis=0)
        X_train = np.concatenate((X_train,nwSpectros), axis=0)
    
    # one-hot encoding voor Y_train
    nrFiles = clzLengths[classes[0]]
    Y_train = np.array((np.ones(nrFiles),np.zeros(nrFiles), np.zeros(nrFiles))).T

    nrFiles = clzLengths[classes[1]]
    Y_train_nw = np.array((np.zeros(nrFiles), np.ones(nrFiles), np.zeros(nrFiles))).T
    Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)

    nrFiles = clzLengths[classes[2]]
    Y_train_nw = np.array((np.zeros(nrFiles), np.zeros(nrFiles), np.ones(nrFiles))).T
    Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)
    
    return X_train, Y_train

### deep learning model

In [8]:
def create_model(layersizes):
    # create model
    model = Sequential()
    model.add(Dense(layersizes[0], input_dim=513, activation='relu'))
    for lsize in layersizes[1:]:
        model.add(Dense(lsize, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
def getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs):
    modelFilename = modelFilePath + baseModelFilename
    for lsize in layers:
        modelFilename = '{}_{}'.format(modelFilename, lsize)
    modelFilename += 'ep{}'.format(nrEpochs)
    modelFilename += '.hd5'
    return modelFilename

In [10]:
def getModelName(baseModelFilename, layers, nrEpochs):
    modelFilename = baseModelFilename
    for lsize in layers:
        modelFilename = '{}_{}'.format(modelFilename, lsize)
    modelFilename += 'ep{}'.format(nrEpochs)
    return modelFilename

In [11]:
def train_model(X_train, Y_train, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = create_model(layers)
    history = timeFunction(lambda: soundModel.fit(X_train,Y_train, epochs=nrEpochs, shuffle=True, verbose=1))
    soundModel.save(getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs))
    plt.plot(history.history['loss'])
    plt.show()

In [12]:
def evaluate_model(X_test, realClasses, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = load_model(getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs))

    # predicted classes
    predictions = soundModel.predict(X_test)
    predClasses = predictions.argmax(axis=1)

    matrix = ConfusionMatrix(classes)
    for vals in zip(realClasses, predClasses):
        matrix.add(int(vals[0]), int(vals[1]), 1)
    LOG(matrix.toString(),True)
    LOG('', True)
    LOG(matrix.toF1String(), True)
    return matrix

In [13]:
def train_and_evaluate_per_epoch(X_train, Y_train, realClasses, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = create_model(layers)
    for epNr in range(1, nrEpochs+1):
        LOG('\n*****************\n* Epoch nr {}\n*****************\n'.format(epNr), True)
        soundModel.fit(X_train,Y_train, epochs=1, shuffle=True, verbose=1)
        soundModel.save(getModelFileName(modelFilePath, baseModelFilename, layers, epNr))
        evaluate_model(X_train, realClasses, layers, epNr, modelFilePath, baseModelFilename)

### functies tbv testen

In [14]:
def createAndStoreTestData(wavFileParts, baseDir, fileDate, micNr, storeFilename, keyName):
    allSpectros = np.array([])
    allClasses = np.array([])

    for wfPt in wavFileParts: #type: WavFilePart
        if not 'Gunshot' in wfPt.getSoundType():
            filename = baseDir + '/{:d}_{:d}_mono{:d}.wav'.format(fileDate, wfPt.fileNr, micNr)
            fs, signal = wv.read(filename)

            classNr = classes.index(wfPt.getSoundType().lower())
            for soundChunk in wfPt.getSoundChunks(micNr):
                startFrame = int(soundChunk[0] * fs)
                endFrame = int(soundChunk[1] * fs)

                sigChunk = signal[startFrame: endFrame]
                freq_array, segment_times, spectrogram = sig.spectrogram(x=sigChunk, fs=fs, nfft=NFFT, noverlap=0)
                if len(allSpectros) == 0:
                    allSpectros = spectrogram.T
                else:
                    allSpectros = np.append(allSpectros, spectrogram.T, axis=0)
                allClasses = np.append(allClasses, classNr * np.ones(len(segment_times)))
                
    storeTestData(allSpectros, allClasses, storeFilename, keyName)


In [15]:
def getTrainDataFromFolders(orgWavDirs):
    fpc = getFilesPerCatFromMultipleDirs(orgWavDirs, baseSrcDir)
    spcs = getSpectrosFromFilesPerCategory(fpc)
    clzLengths = getClassLengths(spcs)
    X_train, Y_train = createTrainDataFromSpectros(spcs, clzLengths)
    # real train classes
    realTrainClasses = np.concatenate((np.zeros(clzLengths[classes[0]]), 
                                  np.ones(clzLengths[classes[1]]), 
                                  2*np.ones(clzLengths[classes[2]])))
    return X_train, Y_train, realTrainClasses


In [16]:
def testModellen(modelFilePath, layerss, micNr, testFileNames, showOverall=True):

    # test data
    keyname = 'mic{}'.format(micNr)

    for layers in layerss:
        for nrEpochs in nrsEpochs:
            overallMatrix = ConfusionMatrix(classes)
            for testFileName in testFileNames:
                testSpecs, testClasses = retrieveTestData(testFileName, keyname)

                LOG_HEADER(getModelName(baseModelFilename, layers, nrEpochs) 
                           + '\n# ' + keyname
                           + '\n# ' + testFileName,
                           True)
                matrix = evaluate_model(testSpecs, testClasses, layers, nrEpochs, modelFilePath, baseModelFilename)
                overallMatrix.addMatrix(matrix.normalizedCopy())
            
            if showOverall:
                overallMatrix = overallMatrix.normalizedCopy()
                LOG_HEADER(getModelName(baseModelFilename, layers, nrEpochs) 
                           + '\n# ' + keyname
                           + '\n# ' + 'testData_Overall',
                           True)
                LOG(overallMatrix.toString(),True)
                LOG('', True)
                LOG(overallMatrix.toF1String(), True)

### Opnieuw tests uitvoeren op oude modellen
* slechts 3 epochs
* dit keer met overall resultaten

In [17]:
baseModelFilename = '20180820_allOrgs_dropout'
modelFilePath = storageFolder
logPrefix = 'Alle Orgs redo met dropout'
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 3
nrsEpochs = [1,2,3]

In [18]:
# Test alle modellen
testModellen(modelFilePath=storageFolder, 
             layerss=layersList, 
             micNr=4, 
             testFileNames=['testData_G428', 'testData_G527', 'testData_Studio'], 
             showOverall=True)                

####################################
#
# 20180820_allOrgs_dropout_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19994     616      29 |  0.97  0.96
voice            999   13177     260 |  0.91  0.96
environment       49      19   13157 |  0.99  0.99
--------------------------------------
prec            0.95    0.95    0.98

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.93
F1 environment: 0.99

####################################
#
# 20180820_allOrgs_dropout_100_20ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8790     123       4 |  0.99  0.97
voice            226    2636       4 |  0.92  0.98
environment        4       0    2487 |  1.00  1.00
--------------------------------------
prec            0.97    0.96    1.00

F1 overall: 0.97
F1 music: 0.98
F1 voice: 0.94
F1 environment: 1.00

######################

                   m       v       e |  sens   acc
music           8776     136       5 |  0.98  0.98
voice            136    2730       0 |  0.95  0.98
environment       12       1    2478 |  0.99  1.00
--------------------------------------
prec            0.98    0.95    1.00

F1 overall: 0.98
F1 music: 0.98
F1 voice: 0.95
F1 environment: 1.00

####################################
#
# 20180820_allOrgs_dropout_400_250_100_20ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          55071    7935     589 |  0.87  0.93
voice            860   36772     272 |  0.97  0.93
environment       20     522   41795 |  0.99  0.99
--------------------------------------
prec            0.98    0.81    0.98

F1 overall: 0.93
F1 music: 0.92
F1 voice: 0.88
F1 environment: 0.98

####################################
#
# 20180820_allOrgs_dropout_400_250_100_20ep2
# mic4
# testData_Overall
#
####################################

 

                   m       v       e |  sens   acc
music          56328    6300     967 |  0.89  0.94
voice           1003   35837    1064 |  0.95  0.94
environment       36     206   42095 |  0.99  0.98
--------------------------------------
prec            0.98    0.85    0.95

F1 overall: 0.93
F1 music: 0.93
F1 voice: 0.89
F1 environment: 0.97

####################################
#
# 20180820_allOrgs_dropout_400_300_200_100_50_20_10ep3
# mic4
# testData_Overall
#
####################################

                   m       v       e |  sens   acc
music           0.95    0.05    0.01 |  0.95  0.96
voice           0.05    0.93    0.01 |  0.93  0.96
environment     0.00    0.00    1.00 |  1.00  0.99
--------------------------------------
prec            0.94    0.95    0.98

F1 overall: 0.96
F1 music: 0.94
F1 voice: 0.94
F1 environment: 0.99

####################################
#
# 20180820_allOrgs_dropout_450_400_350_300_250_200_150_100_50_21ep1
# mic4
# testData_G428
#
########

In [19]:
# G428
baseModelFilename = '20180820_orgG428_dropout'
modelFilePath = storageFolder
logPrefix = 'G428 model redo met dropout'
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 3
nrsEpochs = [1,2,3]

# Test alle modellen
testModellen(modelFilePath=storageFolder, 
             layerss=layersList, 
             micNr=4, 
             testFileNames=['testData_G428', 'testData_G527', 'testData_Studio'], 
             showOverall=True)  

####################################
#
# 20180820_orgG428_dropout_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19282    1303      54 |  0.93  0.97
voice            227   13902     307 |  0.96  0.96
environment       25      53   13147 |  0.99  0.99
--------------------------------------
prec            0.99    0.91    0.97

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.94
F1 environment: 0.98

####################################
#
# 20180820_orgG428_dropout_100_20ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8517     390      10 |  0.96  0.97
voice             97    2760       9 |  0.96  0.97
environment        1       1    2489 |  1.00  1.00
--------------------------------------
prec            0.99    0.88    0.99

F1 overall: 0.96
F1 music: 0.97
F1 voice: 0.92
F1 environment: 1.00

######################

                   m       v       e |  sens   acc
music           8588     305      24 |  0.96  0.97
voice             62    2802       2 |  0.98  0.97
environment        3      10    2478 |  0.99  1.00
--------------------------------------
prec            0.99    0.90    0.99

F1 overall: 0.97
F1 music: 0.98
F1 voice: 0.94
F1 environment: 0.99

####################################
#
# 20180820_orgG428_dropout_400_250_100_20ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          40584   20560    2451 |  0.64  0.84
voice            236   36443    1225 |  0.96  0.82
environment       11    4432   37894 |  0.90  0.94
--------------------------------------
prec            0.99    0.59    0.91

F1 overall: 0.80
F1 music: 0.78
F1 voice: 0.73
F1 environment: 0.90

####################################
#
# 20180820_orgG428_dropout_400_250_100_20ep2
# mic4
# testData_Overall
#
####################################

 

                   m       v       e |  sens   acc
music          40990   20744    1861 |  0.64  0.84
voice            335   36481    1088 |  0.96  0.83
environment       10    2786   39541 |  0.93  0.96
--------------------------------------
prec            0.99    0.61    0.93

F1 overall: 0.82
F1 music: 0.78
F1 voice: 0.75
F1 environment: 0.93

####################################
#
# 20180820_orgG428_dropout_400_300_200_100_50_20_10ep3
# mic4
# testData_Overall
#
####################################

                   m       v       e |  sens   acc
music           0.85    0.14    0.01 |  0.85  0.93
voice           0.05    0.94    0.01 |  0.94  0.92
environment     0.00    0.03    0.97 |  0.97  0.98
--------------------------------------
prec            0.94    0.85    0.98

F1 overall: 0.92
F1 music: 0.89
F1 voice: 0.89
F1 environment: 0.97

####################################
#
# 20180820_orgG428_dropout_450_400_350_300_250_200_150_100_50_21ep1
# mic4
# testData_G428
#
########

In [20]:
# G527
baseModelFilename = '20180820_orgG527_dropout'
modelFilePath = storageFolder
logPrefix = 'G527 model redo met dropout'
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 3
nrsEpochs = [1,2,3]

# Test alle modellen
testModellen(modelFilePath=storageFolder, 
             layerss=layersList, 
             micNr=4, 
             testFileNames=['testData_G428', 'testData_G527', 'testData_Studio'], 
             showOverall=True)  

####################################
#
# 20180820_orgG527_dropout_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19284    1305      50 |  0.93  0.96
voice            257   13726     453 |  0.95  0.96
environment      176      65   12984 |  0.98  0.98
--------------------------------------
prec            0.98    0.91    0.96

F1 overall: 0.95
F1 music: 0.96
F1 voice: 0.93
F1 environment: 0.97

####################################
#
# 20180820_orgG527_dropout_100_20ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8596     313       8 |  0.96  0.97
voice             43    2810      13 |  0.98  0.97
environment        2       0    2489 |  1.00  1.00
--------------------------------------
prec            0.99    0.90    0.99

F1 overall: 0.97
F1 music: 0.98
F1 voice: 0.94
F1 environment: 1.00

######################

                   m       v       e |  sens   acc
music           8761     130      26 |  0.98  0.97
voice            210    2655       1 |  0.93  0.98
environment        1       0    2490 |  1.00  1.00
--------------------------------------
prec            0.98    0.95    0.99

F1 overall: 0.97
F1 music: 0.98
F1 voice: 0.94
F1 environment: 0.99

####################################
#
# 20180820_orgG527_dropout_400_250_100_20ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          46628   13819    3148 |  0.73  0.88
voice            115   36084    1705 |  0.95  0.89
environment        8     543   41786 |  0.99  0.96
--------------------------------------
prec            1.00    0.72    0.90

F1 overall: 0.87
F1 music: 0.85
F1 voice: 0.82
F1 environment: 0.94

####################################
#
# 20180820_orgG527_dropout_400_250_100_20ep2
# mic4
# testData_Overall
#
####################################

 

                   m       v       e |  sens   acc
music          50288   11463    1844 |  0.79  0.90
voice           1260   34567    2077 |  0.91  0.89
environment       19     908   41410 |  0.98  0.97
--------------------------------------
prec            0.98    0.74    0.91

F1 overall: 0.88
F1 music: 0.87
F1 voice: 0.81
F1 environment: 0.94

####################################
#
# 20180820_orgG527_dropout_400_300_200_100_50_20_10ep3
# mic4
# testData_Overall
#
####################################

                   m       v       e |  sens   acc
music           0.91    0.07    0.01 |  0.91  0.94
voice           0.08    0.90    0.03 |  0.90  0.94
environment     0.02    0.01    0.97 |  0.97  0.98
--------------------------------------
prec            0.91    0.91    0.96

F1 overall: 0.93
F1 music: 0.91
F1 voice: 0.90
F1 environment: 0.97

####################################
#
# 20180820_orgG527_dropout_450_400_350_300_250_200_150_100_50_21ep1
# mic4
# testData_G428
#
########

In [21]:
# Studio
baseModelFilename = '20180820_orgStudio_dropout'
modelFilePath = storageFolder
logPrefix = 'Studio model redo met dropout'
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 3
nrsEpochs = [1,2,3]

# Test alle modellen
testModellen(modelFilePath=storageFolder, 
             layerss=layersList, 
             micNr=4, 
             testFileNames=['testData_G428', 'testData_G527', 'testData_Studio'], 
             showOverall=True)  

####################################
#
# 20180820_orgStudio_dropout_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          20479     154       6 |  0.99  0.86
voice           4564    9645     227 |  0.67  0.90
environment     2029      13   11183 |  0.85  0.95
--------------------------------------
prec            0.76    0.98    0.98

F1 overall: 0.85
F1 music: 0.86
F1 voice: 0.80
F1 environment: 0.91

####################################
#
# 20180820_orgStudio_dropout_100_20ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8899      18       0 |  1.00  0.92
voice            830    2027       9 |  0.71  0.94
environment      255       0    2236 |  0.90  0.98
--------------------------------------
prec            0.89    0.99    1.00

F1 overall: 0.90
F1 music: 0.94
F1 voice: 0.83
F1 environment: 0.94

##################

                   m       v       e |  sens   acc
music           8911       5       1 |  1.00  0.90
voice           1074    1788       4 |  0.62  0.92
environment      369       0    2122 |  0.85  0.97
--------------------------------------
prec            0.86    1.00    1.00

F1 overall: 0.87
F1 music: 0.92
F1 voice: 0.77
F1 environment: 0.92

####################################
#
# 20180820_orgStudio_dropout_400_250_100_20ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          59686    3775     134 |  0.94  0.96
voice           1321   36174     409 |  0.95  0.96
environment       99     402   41836 |  0.99  0.99
--------------------------------------
prec            0.98    0.90    0.99

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.92
F1 environment: 0.99

####################################
#
# 20180820_orgStudio_dropout_400_250_100_20ep2
# mic4
# testData_Overall
#
###################################

                   m       v       e |  sens   acc
music          58161    5309     125 |  0.91  0.96
voice            282   37169     453 |  0.98  0.96
environment       83     383   41871 |  0.99  0.99
--------------------------------------
prec            0.99    0.87    0.99

F1 overall: 0.95
F1 music: 0.95
F1 voice: 0.92
F1 environment: 0.99

####################################
#
# 20180820_orgStudio_dropout_400_300_200_100_50_20_10ep3
# mic4
# testData_Overall
#
####################################

                   m       v       e |  sens   acc
music           0.96    0.04    0.00 |  0.96  0.91
voice           0.12    0.87    0.01 |  0.87  0.94
environment     0.11    0.00    0.89 |  0.89  0.96
--------------------------------------
prec            0.81    0.96    0.99

F1 overall: 0.91
F1 music: 0.88
F1 voice: 0.91
F1 environment: 0.94

####################################
#
# 20180820_orgStudio_dropout_450_400_350_300_250_200_150_100_50_21ep1
# mic4
# testData_G428
#
####

In [22]:
from parseExperimentResults import parseRedoExperiment